In [ ]:
import pandas as pd
import numpy as np
import os

from datetime import datetime

In [ ]:
filename = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Current_G3x_con.csv'
df = pd.read_csv(filename, sep = ',')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('#', 'num')

filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/GDY_CON_Incentive.txt'
df_tierMap = pd.read_csv(filename2, sep = '\t')
df.shape

In [ ]:
# df_tierMap

In [ ]:
# df.columns

In [ ]:
# Logic to check if qualify for EOY incentive (1st 3 quarters)
filenameEOQ = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/EOQ'
fileList = os.listdir(filenameEOQ)
df_files = pd.Series(fileList)

tfQ1 = df_files.str.contains('Q1.txt', regex=False)
tfQ2 = df_files.str.contains('Q2.txt', regex=False)
tfQ3 = df_files.str.contains('Q3.txt', regex=False)
tfQ4 = df_files.str.contains('Q4.txt', regex=False)

Q1_file = df_files[tfQ1].reset_index(drop=True)
if not Q1_file.empty:
    foo = (filenameEOQ + '/' + Q1_file[0])
    q1 = pd.read_csv(foo, sep = '\t')
    q1_select = q1.iloc[:,[13,14,16,17,18]]
Q2_file = df_files[tfQ2].reset_index(drop=True)
if not Q2_file.empty:
    foo = (filenameEOQ + '/' + Q2_file[0])
    q2 = pd.read_csv(foo, sep = '\t')
    q2_select = q2.iloc[:,[13,14,16,17,18]]
Q3_file = df_files[tfQ3].reset_index(drop=True)
if not Q3_file.empty:
    foo = (filenameEOQ + '/' + Q3_file[0])
    q3 = pd.read_csv(foo, sep = '\t')
    q3_select = q1.iloc[:,[13,14,16,17,18]]
Q4_file = df_files[tfQ4].reset_index(drop=True)
if not Q4_file.empty:
    foo = (filenameEOQ + '/' + Q4_file[0])
    q4 = pd.read_csv(foo, sep = '\t')
    q4_select = q4.iloc[:,[13,14,16,17,18]]

In [ ]:
Q1_file.empty+Q1_file.empty+Q1_file.empty+Q1_file.empty

In [ ]:
Q1_file.empty + Q2_file.empty + Q3_file.empty + Q4_file.empty 

In [ ]:
currentYear = datetime.now().year

In [ ]:
tup_now = datetime.now().timetuple()

In [ ]:
ts_currentData = os.path.getctime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData)
# print(dt_currentData)
ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_incentive)

In [ ]:
calculatedData = pd.DataFrame([])
for ii in range(df.shape[0]):

    singleCusty = df.iloc[[ii]]
    singleCusty.reset_index(inplace=True)
    # singleCusty

    singleCusty['QTDsum'] = singleCusty.Goodyear_Core_Net_Purchases_QTD + singleCusty.Kelly_Net_Purchases_QTD + singleCusty.Other_Net_Purchases_QTD
    singleCusty_select = singleCusty[
        ['G3X_CO_Name','num_Locations','Goodyear_Core_Net_Purchases_QTD','Kelly_Net_Purchases_QTD','Other_Net_Purchases_QTD','QTDsum']]
    singleCusty_select = singleCusty_select.rename(index=str, columns={
        'G3X_CO_Name':'Name','num_Locations':'numLocs','Goodyear_Core_Net_Purchases_QTD':'goodyearQTD','Kelly_Net_Purchases_QTD':'kellyQTD','Other_Net_Purchases_QTD':'dunlopQTD'})
    excel_date = singleCusty.Last_Transaction
    # excel_date

    date_lt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + excel_date - 2)
    tt = date_lt.timetuple()
        
    if  tt.tm_year == currentYear:

        if (tt.tm_mon == 1) | (tt.tm_mon == 2) | (tt.tm_mon == 3):
            if ((tt.tm_year % 400 == 0) | (tt.tm_year % 4 == 0)) & (tt.tm_year % 100 != 0):
                quarter = 1
                daysINqtr = 91
                print('Leap year')
            else:
                quarter = 1
                daysINqtr = 90           
        elif (tt.tm_mon == 4) | (tt.tm_mon == 5) | (tt.tm_mon == 6):
            quarter = 2
            daysINqtr = 91
            print(quarter)
            break # for testing            
        elif (tt.tm_mon == 7) | (tt.tm_mon == 8) | (tt.tm_mon == 9):
            quarter = 3
            daysINqtr = 92
            print(quarter)
            break # for testing
        else:
            quarter = 4
            daysINqtr = 92
            print(quarter)
            break # for testing
        
        numDays = tt.tm_yday # get number of days

        estimatedTires = np.floor((singleCusty.QTDsum/(numDays/daysINqtr))).values
        numLocations = singleCusty.num_Locations.values

#         if   (estimatedTires >= (df_tierMap.iloc[1,4] * numLocations)) & (estimatedTires <= (df_tierMap.iloc[1,5] * numLocations)):
#             tierType = 1
#         elif (estimatedTires >= (df_tierMap.iloc[2,4] * numLocations)) & (estimatedTires <= (df_tierMap.iloc[2,5] * numLocations)):
#             tierType = 2
#         elif (estimatedTires >= (df_tierMap.iloc[3,4] * numLocations)) & (estimatedTires <= (df_tierMap.iloc[3,5] * numLocations)):
#             tierType = 3
#         elif (estimatedTires >= (df_tierMap.iloc[4,4] * numLocations)):
#             tierType = 4
#         else:
#             tierType = 0
        
#         if tierType < 4:
#             tierUP = tierType + 1
#         else:
#             tierUP = 4
        
        if   (estimatedTires >= (df_tierMap.iloc[1,4] )) & (estimatedTires <= (df_tierMap.iloc[1,5])):
            tierType = 1
        elif (estimatedTires >= (df_tierMap.iloc[2,4] )) & (estimatedTires <= (df_tierMap.iloc[2,5])):
            tierType = 2
        elif (estimatedTires >= (df_tierMap.iloc[3,4] )) & (estimatedTires <= (df_tierMap.iloc[3,5])):
            tierType = 3
        elif (estimatedTires >= (df_tierMap.iloc[4,4])):
            tierType = 4
        else:
            tierType = 0
        
        if tierType < 4:
            tierUP = tierType + 1
        else:
            tierUP = 4
        
        # floor to underestimate estimated tires
        goodYR_est_tires = np.floor((singleCusty.Goodyear_Core_Net_Purchases_QTD/(numDays/daysINqtr)).values)
        goodYR_est_tires_rebate = goodYR_est_tires * df_tierMap.iloc[tierType,1]
        goodYR_prop = singleCusty_select.goodyearQTD/singleCusty_select.QTDsum
        # ceil to make sure the customer surpasses the minimal 1up tire threshold
        goodYR_1up_tires = np.ceil((goodYR_prop*df_tierMap.iloc[tierUP,4]*numLocations).values)
        goodYR_1up_tires_rebate = goodYR_1up_tires * df_tierMap.iloc[tierUP,1]
        
        kelly_est_tires = np.floor((singleCusty.Kelly_Net_Purchases_QTD/(numDays/daysINqtr)).values)
        kelly_est_tires_rebate = kelly_est_tires * df_tierMap.iloc[tierType,2]
        kelly_prop = singleCusty_select.kellyQTD/singleCusty_select.QTDsum
        kelly_1up_tires = np.ceil((kelly_prop*df_tierMap.iloc[tierUP,4]*numLocations).values)
        kelly_1up_tires_rebate = kelly_1up_tires * df_tierMap.iloc[tierUP,2]
        
        dunlop_est_tires = np.floor((singleCusty.Other_Net_Purchases_QTD/(numDays/daysINqtr)).values)
        dunlop_est_tires_rebate = dunlop_est_tires * df_tierMap.iloc[tierType,3]
        dunlop_prop = singleCusty_select.dunlopQTD/singleCusty_select.QTDsum
        dunlop_1up_tires = np.ceil((dunlop_prop*df_tierMap.iloc[tierUP,4]*numLocations).values)
        dunlop_1up_tires_rebate = dunlop_1up_tires * df_tierMap.iloc[tierUP,3]
        
        total_est_tires = goodYR_est_tires + kelly_est_tires + dunlop_est_tires
        total_est_dollars = goodYR_est_tires_rebate+kelly_est_tires_rebate+dunlop_est_tires_rebate
        total_1up_dollars = goodYR_1up_tires_rebate+kelly_1up_tires_rebate+dunlop_1up_tires_rebate
        total_1up_tires = goodYR_1up_tires+kelly_1up_tires+dunlop_1up_tires
        
        if estimatedTires > total_1up_tires:
            goodYR_1up_tires = goodYR_est_tires
            kelly_1up_tires = kelly_est_tires
            dunlop_1up_tires = dunlop_est_tires
            total_1up_tires = estimatedTires
            
            goodYR_1up_tires_rebate = goodYR_est_tires_rebate
            kelly_1up_tires_rebate = kelly_est_tires_rebate
            dunlop_1up_tires_rebate = dunlop_est_tires_rebate
            total_1up_dollars = total_est_dollars
        
#         range_tires = abs(singleCusty_select.QTDsum-df_tierMap.iloc[tierUP,4])
#         added_est_dollars = ((goodYR_est_tires*df_tierMap.iloc[tierUP,1])+(
#             kelly_est_tires*df_tierMap.iloc[tierUP,2])+(
#             dunlop_est_tires*df_tierMap.iloc[tierUP,3]))

        range_dollars_ll = ((total_1up_tires*df_tierMap.iloc[tierUP,3]))#.values
        range_dollars_ul = ((total_1up_tires*df_tierMap.iloc[tierUP,1]))#.values
        
###########################################################################################################################        
#         print(singleCusty_select)
# ###########################################################################################################################     
#         print('\nEstimated Tier:', [tierType])
        
#         print('\nEstimated Tires ~ Goodyear:', goodYR_est_tires)
#         print('Estimated Tires ~ Kelly:', kelly_est_tires)        
#         print('Estimated Tires ~ Dunlop:', dunlop_est_tires)
#         print('-------------------------------------')
#         print('Estimated Tires ~ Total:',estimatedTires)
        
#         print('\nEstimated Dollars ~ Goodyear:',goodYR_est_tires_rebate)
#         print('Estimated Dollars ~ Kelly:',kelly_est_tires_rebate)
#         print('Estimated Dollars ~ Dunlop:',dunlop_est_tires_rebate)
#         print('-------------------------------------')
#         print('Estimated Dollars ~ Total:',total_est_dollars)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################
#         print('Estimated 1up Tier:', [tierUP])
        
#         print('\n1up Tires ~ Goodyear:', goodYR_1up_tires)
#         print('1up Tires ~ Kelly:', kelly_1up_tires)        
#         print('1up Tires ~ Dunlop:', dunlop_1up_tires)
#         print('-------------------------------------')
#         print('Estimated 1up Tires ~ Total:',total_1up_tires)
        
#         print('\n1up Dollars ~ Goodyear:',goodYR_1up_tires_rebate)
#         print('1up Dollars ~ Kelly:',kelly_1up_tires_rebate)
#         print('1up Dollars ~ Dunlop:',dunlop_1up_tires_rebate)
#         print('-------------------------------------')
#         print('1up Dollars ~ Total:',total_1up_dollars)
#         print('\n-----------------------------------------------------------------------------------------------------------------------\n')
# ###########################################################################################################################        
#         print('Range_ll:',range_dollars_ll)
#         print('Range_ul:',range_dollars_ul, '\n')
# ###########################################################################################################################
#         print(tt, '\n' 'indexNumber:', ii, '\n')
###########################################################################################################################
##### ADD QUARTER AND GRAIN TO EACH PROGRAM
        tempData = pd.DataFrame(
            {'estimated_tier':tierType,
             'GY_est_tires':goodYR_est_tires, 'Kelly_est_tires':kelly_est_tires, 'Dunlop_est_tires':dunlop_est_tires, 'current_est_tires':total_est_tires*numLocations,
             'GY_est_rebate':goodYR_est_tires_rebate, 'Kelly_est_rebate':kelly_est_tires_rebate, 'Dunlop_est_rebate':dunlop_est_tires_rebate,
             'total_est_rebate': total_est_dollars,'Up_tier':tierUP,
             'GY_up_tires':goodYR_1up_tires, 'Kelly_up_tires':kelly_1up_tires, 'Dunlop_up_tires':dunlop_1up_tires, 'total_up_tires':total_1up_tires,
             'GY_up_rebate':goodYR_1up_tires_rebate, 'Kelly_up_rebate':kelly_1up_tires_rebate, 'Dunlop_up_rebate':dunlop_1up_tires_rebate,
             'total_up_rebate':total_1up_dollars,
             'range_lower':range_dollars_ll, 'range_upper':range_dollars_ul,'period': quarter, 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
    else:
#         print('\nJUNK:', tt,'\n', 'indexNumber:', ii)
        range_dollars_ll = (df_tierMap.iloc[1,3] * df_tierMap.iloc[1,4])
        range_dollars_ul = (df_tierMap.iloc[1,1] * df_tierMap.iloc[1,4]) 
        tempData = pd.DataFrame(
            {'estimated_tier':0,
             'GY_est_tires':0, 'Kelly_est_tires':0, 'Dunlop_est_tires':0, 'current_est_tires':0, 
             'GY_est_rebate':0, 'Kelly_est_rebate':0, 'Dunlop_est_rebate':0,
             'total_est_rebate': 0, 'Up_tier':1,
             'GY_up_tires':0, 'Kelly_up_tires':0, 'Dunlop_up_tires':0, 'total_up_tires':0,
             'GY_up_rebate':0, 'Kelly_up_rebate':0, 'Dunlop_up_rebate':0,
             'total_up_rebate':0,
             'range_lower':[range_dollars_ll], 'range_upper':[range_dollars_ul],'period': quarter, 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)
finalData = pd.concat([calculatedData,df], axis=1)

In [ ]:
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_G3X_CON.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_G3X_CON_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)

In [ ]:
# path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
# writeFilename = path + 'cmp_G3X.csv'
# finalData.to_csv(writeFilename, index=False)
# writeFilename_archive = path + 'csv_archive/' + str(dt_currentData.year) + '_' + str(dt_currentData.month) + '_' + str(dt_currentData.day) + '_' + 'CMP_G3X.csv'
# finalData.to_csv(writeFilename_archive, index=False)